In [183]:
import pandas as pd

stats_2023 = pd.read_csv("../../data/preprocess/pre-processed.csv")
stats_2023.drop(columns=["Profit", "Selling", "Revenue"], inplace=True)
stats_2023.dtypes

Field ID           int64
Field Name        object
Crop ID            int64
Crop Name         object
Crop Type         object
Planting Area    float64
Season ID          int64
Season            object
Field Type        object
Field Area       float64
Per Yield        float64
Per Cost         float64
Per Price        float64
Yield            float64
Cost             float64
dtype: object

# 初始化

In [184]:
import pulp
import numpy as np

# 创建线性规划问题
prob1 = pulp.LpProblem("Crop_Optimization", pulp.LpMaximize)
prob2 = pulp.LpProblem("Crop_Optimization", pulp.LpMaximize)

# 定义辅助变量

In [185]:
fields_id = stats_2023["Field ID"].unique()  # 所有的地块
crops_id = stats_2023["Crop ID"].unique()  # 所有的作物
seasons_id = stats_2023["Season ID"].unique()  # 所有的时节
years = range(2024, 2031)  # 要优化的年份


def pop_ndarray(arr, element_to_pop):
    index = np.where(arr == element_to_pop)[0]
    if index.size > 0:
        arr = np.delete(arr, index)
    return arr


# A类粮食作物的集合
grains_A = stats_2023[stats_2023["Crop Type"] == "粮食"]["Crop ID"].unique()
grains_A = pop_ndarray(grains_A, 16)

# B类粮食作物的集合
grains_B = np.array([16])

# A类蔬菜作物的集合
vege_A = stats_2023[stats_2023["Crop Type"] == "蔬菜"]["Crop ID"].unique()
for i in range(3):
    vege_A = pop_ndarray(vege_A, 35 + i)

# B类蔬菜作物的集合
vege_B = np.array([35, 36, 37])

# 食用菌作物的集合
mush = stats_2023[stats_2023["Crop Type"] == "食用菌"]["Crop ID"].unique()

# 豆类作物的集合
beans = stats_2023[
    (stats_2023["Crop Type"] == "粮食（豆类）")
    | (stats_2023["Crop Type"] == "蔬菜（豆类）")
]["Crop ID"].unique()

# 第 i 个地块的类型（如平旱地、梯田、山坡地、智能大棚、普通大棚、水浇地）
t_i = {
    i: stats_2023[stats_2023["Field ID"] == i]["Field Type"].values[0]
    for i in fields_id
}

# 第 i 个地块在第 s 季可种植的作物集合
T_hat_i_s: dict[tuple : np.ndarray] = {}
water_field_first_season_choice = {}  # 记录水浇地在第一季的选择


# 随机获取水浇地在第一季的选择
def get_first_season_choice():
    import random

    return random.choice(list(np.concatenate([grains_B, vege_A])))


for i in fields_id:
    if t_i[i] in ["平旱地", "梯田", "山坡地"]:
        T_hat_i_s[(i, 1)] = grains_A
        T_hat_i_s[(i, 2)] = np.array([])
    elif t_i[i] in ["水浇地"]:
        first_season_choice = get_first_season_choice()
        water_field_first_season_choice[i] = first_season_choice
        if first_season_choice in grains_B:
            T_hat_i_s[(i, 1)] = grains_B
            T_hat_i_s[(i, 2)] = np.array([])
        elif first_season_choice in vege_A:
            T_hat_i_s[(i, 1)] = vege_A
            T_hat_i_s[(i, 2)] = vege_B
    elif t_i[i] in ["普通大棚"]:
        T_hat_i_s[(i, 1)] = vege_A
        T_hat_i_s[(i, 2)] = mush
    elif t_i[i] in ["智慧大棚"]:
        T_hat_i_s[(i, 1)] = vege_A
        T_hat_i_s[(i, 2)] = vege_A

S = {}  # 期望产量
Y = {}  # 单位面积产量
P = {}  # 售价
C = {}  # 单位面积成本

for j in crops_id:
    # 所有参数与 2023 年的保持一致
    for k in years:
        S[(j, k)] = stats_2023.groupby(["Crop ID"])["Yield"].agg("sum").values[j - 1]
        Y[(j, k)] = stats_2023[stats_2023["Crop ID"] == j]["Per Yield"].values[0]
        P[(j, k)] = stats_2023[stats_2023["Crop ID"] == j]["Per Price"].values[0]
        C[(j, k)] = stats_2023[stats_2023["Crop ID"] == j]["Per Cost"].values[0]

non_tent_field = [i for i in fields_id if not t_i[i].endswith("大棚")]
tent_field = [i for i in fields_id if t_i[i].endswith("大棚")]

# 定义决策变量

In [186]:
# A^1_{ijks}：第 k 年第 s 季度的第 i 块地，第 j 作物的种植面积，若为不为大棚地块则为 0
# A^2_{ijks}：第 k 年第 s 季度的第 i 块地，第 j 作物的种植面积，若为大棚地块则为 0
# A_{ijks}：第 k 年第 s 季度的第 i 块地，第 j 作物的种植面积，若为大棚地块则为 0

A1_ijks = pulp.LpVariable.dicts(
    "A1",
    [
        (i, j, k, s)
        for i in fields_id
        for j in crops_id
        for k in years
        for s in seasons_id
    ],
    lowBound=0,
    cat="Continuous",
)

A2_ijks = pulp.LpVariable.dicts(
    "A2",
    [
        (i, j, k, s)
        for i in fields_id
        for j in crops_id
        for k in years
        for s in seasons_id
    ],
    lowBound=0,
    cat="Continuous",
)

# Rename A1 and A2
for i in fields_id:
    for j in crops_id:
        for k in years:
            for s in seasons_id:
                A1_ijks[(i, j, k, s)].name = f"A1_{i}_{j}_{k}_{s}"
                A2_ijks[(i, j, k, s)].name = f"A2_{i}_{j}_{k}_{s}"


for i in fields_id:
    for j in crops_id:
        for k in years:
            for s in seasons_id:
                if i in non_tent_field:
                    A2_ijks[(i, j, k, s)].upBound = 0
                elif i in tent_field:
                    A1_ijks[(i, j, k, s)].upBound = 0

# k = 2023 的决策变量已知，且假设 2022 年的情况与 2023 年相同
for i in fields_id:
    for j in crops_id:
        for s in seasons_id:
            # 获取匹配的数据行
            matching_rows = stats_2023[
                (stats_2023["Field ID"] == i)
                & (stats_2023["Crop ID"] == j)
                & (stats_2023["Season"] == s)
            ]

            area_sum = (
                matching_rows["Planting Area"].values[0]
                if not matching_rows.empty
                else 0
            )

            if i in tent_field:
                A1_ijks[(i, j, 2023, s)] = pulp.LpVariable(
                    f"A1_{i}_{j}_2023_{s}",
                    lowBound=area_sum,
                    upBound=area_sum,
                    cat="Continuous",
                )
                A2_ijks[(i, j, 2023, s)] = pulp.LpVariable(
                    f"A2_{i}_{j}_2023_{s}",
                    lowBound=0,
                    upBound=0,
                    cat="Continuous",
                )
                A1_ijks[(i, j, 2023, s)].setInitialValue(area_sum)
                A2_ijks[(i, j, 2023, s)].setInitialValue(0)
            elif i in non_tent_field:
                A1_ijks[(i, j, 2023, s)] = pulp.LpVariable(
                    f"A1_{i}_{j}_2023_{s}",
                    lowBound=0,
                    upBound=0,
                    cat="Continuous",
                )
                A2_ijks[(i, j, 2023, s)] = pulp.LpVariable(
                    f"A2_{i}_{j}_2023_{s}",
                    lowBound=area_sum,
                    upBound=area_sum,
                    cat="Continuous",
                )
                A1_ijks[(i, j, 2023, s)].setInitialValue(0)
                A2_ijks[(i, j, 2023, s)].setInitialValue(area_sum)

            # 2022 年的情况与 2023 年相同
            A1_ijks[(i, j, 2022, s)] = A1_ijks[(i, j, 2023, s)]
            A2_ijks[(i, j, 2022, s)] = A2_ijks[(i, j, 2023, s)]


# A_{ijk} = \sum_s A^2_{ijks} + A^1_{ij(k-1)2} + A^1_{ijks}
A_ijk: dict[tuple : pulp.LpVariable] = {}
for i in fields_id:
    for j in crops_id:
        for k in years:
            # Pre-compute sums outside the inner loop
            sum_A2 = pulp.lpSum([A2_ijks[i, j, k, s] for s in seasons_id])
            sum_A1 = pulp.lpSum([A1_ijks[i, j, k, s] for s in seasons_id])
            A_ijk[(i, j, k)] = sum_A2 + sum_A1 + A1_ijks[i, j, k - 1, 2]

# A_ijks = A^1_{ijks} + A^2_{ijks}
A_ijks: dict[tuple : pulp.LpVariable] = {}
for i in fields_id:
    for j in crops_id:
        for k in [2023, *years]:
            for s in seasons_id:
                A_ijks[(i, j, k, s)] = A1_ijks[(i, j, k, s)] + A2_ijks[(i, j, k, s)]

A_star = {}  # 地块面积
area_temp = {}

for i in fields_id:
    area_temp[i] = stats_2023[stats_2023["Field ID"] == i]["Field Area"].values[0]

for i in area_temp:
    A_star[i] = pulp.LpVariable(
        f"A_star_{i}",
        lowBound=area_temp[i],
        upBound=area_temp[i],
        cat="Continuous",
    )
    A_star[i].setInitialValue(area_temp[i])

# 定义目标函数

In [187]:
# Loosely check inequality and not abs function used
def min_loose(a, b):
    return a if a <= b else b


# Loosely check inequality and not abs function used
def max_loose(a, b):
    return a if a >= b else b


# L_1 = \sum_{ijk} \left( \min \left( S_{jk}, Y_{jk} \cdot A_{ijk} \right) \cdot P_{jk} - C_{jk} \cdot A_{ijk} \right)
L1 = pulp.lpSum(
    [
        (
            min_loose(S[(j, k)], Y[(j, k)] * A_ijk[(i, j, k)]) * P[(j, k)]
            - C[(j, k)] * A_ijk[(i, j, k)]
        )
        for i in fields_id
        for j in crops_id
        for k in years
    ]
)

# L_2 = \sum_{ijk} \left( Y_{jk} \cdot A_{ijk} \cdot P_{jk} - C_{jk} \cdot A_{ijk} - 0.5 \cdot \max \left( 0, Y_{jk} \cdot A_{ijk} - S_{jk} \right) \cdot P_{jk} \right)
L2 = pulp.lpSum(
    [
        (
            Y[(j, k)] * A_ijk[(i, j, k)] * P[(j, k)]
            - C[(j, k)] * A_ijk[(i, j, k)]
            - 0.5 * max_loose(0, Y[(j, k)] * A_ijk[(i, j, k)] - S[(j, k)]) * P[(j, k)]
        )
        for i in fields_id
        for j in crops_id
        for k in years
    ]
)

# 设置目标函数
prob1 += L1
prob2 += L2

# 约束条件一：\sum_j A_{ijks} \leq A_i^* \quad \forall i,k \text{ and } j \in \hat{T}_{is}
for i in fields_id:
    for k in years:
        for s in seasons_id:
            if not T_hat_i_s[(i, s)].size == 0:
                prob1 += (
                    pulp.lpSum([A_ijks[(i, j, k, s)] for j in T_hat_i_s[(i, s)]])
                    <= A_star[i]
                )
                prob2 += (
                    pulp.lpSum([A_ijks[(i, j, k, s)] for j in T_hat_i_s[(i, s)]])
                    <= A_star[i]
                )

# 约束条件二：A_{ij(k-1)s}+A_{ijks} \leq min(A_{ij(k-1)s},A_{ijks}) \quad \forall i,k \text{ and } j \in \hat{T}_{is}
for i in fields_id:
    for k in years:
        for s in seasons_id:
            if not T_hat_i_s[(i, s)].size == 0:
                prob1 += A_ijks[(i, j, k - 1, s)] + A_ijks[(i, j, k, s)] <= min_loose(
                    A_ijks[(i, j, k - 1, s)], A_ijks[(i, j, k, s)]
                )
                prob2 += A_ijks[(i, j, k - 1, s)] + A_ijks[(i, j, k, s)] <= min_loose(
                    A_ijks[(i, j, k - 1, s)], A_ijks[(i, j, k, s)]
                )

# 约束条件三：A_{ij(k-1)2} \leq A_{ij(k-1)1} \quad \forall i,k \text{ and } j \in \hat{T}_{is}
for i in fields_id:
    for k in years:
        for s in seasons_id:
            if not T_hat_i_s[(i, s)].size == 0:
                prob1 += A_ijks[(i, j, k - 1, 2)] <= A_ijks[(i, j, k - 1, 1)]
                prob2 += A_ijks[(i, j, k - 1, 2)] <= A_ijks[(i, j, k - 1, 1)]

M = 0.25
# 约束条件四：A_{ijks}^{n} \geq M \times A_i^*  \quad \text{if } A_{ijks}^{n} \neq 0 \qquad \forall i,k \text{ and } j \in \hat{T}_{is},n \in \{1,2\}
for i in fields_id:
    for k in years:
        for s in seasons_id:
            if not T_hat_i_s[(i, s)].size == 0:
                for n in [1, 2]:
                    if A_ijks[(i, j, k, s)] != 0:
                        prob1 += A_ijks[(i, j, k, s)] >= M * A_star[i]
                        prob2 += A_ijks[(i, j, k, s)] >= M * A_star[i]

# 模型求解

In [188]:
prob1.solve()
prob2.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/zivmax/CUMCM_2024/.venv/lib/python3.12/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/b12ce73c69e2428fb4c2307e0770d6e9-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /tmp/b12ce73c69e2428fb4c2307e0770d6e9-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 2875 COLUMNS
At line 87284 RHS
At line 90155 BOUNDS
At line 123582 ENDATA
Problem MODEL has 2870 rows, 64422 columns and 19054 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve determined that the problem was infeasible with tolerance of 1e-08
Analysis indicates model infeasible or unbounded
1 infeasibilities
Analysis indicates model infeasible or unbounded
Perturbing problem by 0.001% of 20000 - largest nonzero change 0.023342348 ( 0.00077807825%) - largest zero change 0
0  Obj -0 Primal inf 4668.9999 (1148)
28  Obj -1606503.6 Prim

-1

# 整理结果

In [189]:
A_ijks_1 = {}

A_ijks_2 = {}

for v in prob1.variables():
    if v.name.startswith("A_star"):
        continue

    if v.name.startswith("A1"):
        v.name.replace("A1_", "")
        A_ijks_1[tuple(v.name.split("_"))] = v.varValue

print(A_ijks_1[(1, 1, 2024, "第一季")])

KeyError: (1, 1, 2024, '第一季')